In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split


Read dataset


In [3]:
dataset = pd.read_csv("output-dataset1.csv")
dataset


,frame.encap_type,frame.time_epoch,frame.len,frame.cap_len,eth.src,eth.dst,ip.version,ip.hdr_len,ip.tos,ip.id,...,ip.checksum,ip.src,ip.dst,ip.len,ip.dsfield,srcport,dstport,length,checksum,quic
0,1,1.645532e+09,75,75,f8:5e:a0:dd:40:4b,9c:9d:7e:1b:50:38,4,20.0,NaN,0x72be,...,0x0000,192.168.31.158,142.250.77.42,61.0,0x00,55723,443,41,0xbca5,0
1,1,1.645532e+09,67,67,9c:9d:7e:1b:50:38,f8:5e:a0:dd:40:4b,4,20.0,NaN,0x0000,...,0x844d,142.250.77.42,192.168.31.158,53.0,0x00,443,55723,33,0xbcd8,0
2,1,1.645532e+09,75,75,f8:5e:a0:dd:40:4b,9c:9d:7e:1b:50:38,4,20.0,NaN,0x858a,...,0x0000,192.168.31.158,142.250.182.202,61.0,0x00,52672,443,41,0x2646,0
3,1,1.645532e+09,67,67,9c:9d:7e:1b:50:38,f8:5e:a0:dd:40:4b,4,20.0,NaN,0x0000,...,0x1aad,142.250.182.202,192.168.31.158,53.0,0x00,443,52672,33,0xd64c,0
4,1,1.645532e+09,292,292,22:59:8b:af:b0:89,01:00:5e:00:00:fb,4,20.0,NaN,0x48a5,...,0x709d,192.168.31.240,224.0.0.251,278.0,0x00,5353,5353,258,0xff99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204837,1,1.645534e+09,54,54,f8:5e:a0:dd:40:4b,9c:9d:7e:1b:50:38,4,20.0,NaN,0xd5e3,...,0x0000,192.168.31.158,23.221.32.227,40.0,0x00,49827,443,20,0x1921,0
204838,1,1.645534e+09,66,66,9c:9d:7e:1b:50:38,f8:5e:a0:dd:40:4b,4,20.0,NaN,0x0000,...,0x5d43,52.32.33.250,192.168.31.158,52.0,0x20,443,49826,32,0xb27d,0
204839,1,1.645534e+09,54,54,f8:5e:a0:dd:40:4b,9c:9d:7e:1b:50:38,4,20.0,NaN,0x6352,...,0x0000,192.168.31.158,52.32.33.250,40.0,0x00,49826,443,20,0x367b,0
204840,1,1.645534e+09,263,263,f8:5e:a0:dd:40:4b,9c:9d:7e:1b:50:38,4,20.0,NaN,0x6353,...,0x0000,192.168.31.158,52.32.33.250,249.0,0x00,49826,443,20,0x374c,0


Preprocess dataset

- Extract quic column data and remove from dataset.
- Remove the string related feilds.
- Remove empty feilds


In [ ]:
def preprocess(dataset):

    # 1. Remove the strings columnns from dataset.
    # eth.src, eth.dst, ip.src, ip.dst, ip.tos, ip.id, ip.flags, ip.checksum, ip.dsfield, checksum
    dataset.drop(
        [
            "eth.src",
            "eth.dst",
            "ip.src",
            "ip.dst",
            "ip.tos",
            "ip.id",
            "ip.flags",
            "ip.checksum",
            "ip.dsfield",
            "checksum",
        ],
        axis=1,
        inplace=True,
    )

    # 2. Remove the invalid rows ie axis=0 from the dataset.
    dataset.dropna(axis=0, inplace=True)

    # 3. After deleting the rows, reset the index column and delete it.
    dataset = dataset.reset_index(drop=True)

    # 4. Get the result(quic) column values and remove dataset.
    targets = list(dataset["quic"].values)
    dataset.drop(dataset.columns[len(dataset.columns) - 1], axis=1, inplace=True)

    print(dataset.info())
    return dataset, targets


initial_size = dataset.size
clean_dataset, targets = preprocess(dataset)
final_size = clean_dataset.size

if clean_dataset.isnull().values.any():
    print("Failed to clean dataset. There are NaN/null values in dataset.")
    nan_cols = [i for i in clean_dataset.columns if clean_dataset[i].isnull().any()]
    print("Columns with null values: ", *nan_cols)
    exit(1)


print(
    "\nDataset size: {0} = {1}, Cleaned dataset size: {2} = {3}, Percentage: {4}%".format(
        dataset.shape,
        initial_size,
        clean_dataset.shape,
        final_size,
        round(100 - (final_size / initial_size) * 100, 2),
    )
)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204727 entries, 0 to 204726
Data columns (total 16 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   frame.encap_type  204727 non-null  int64  
 1   frame.time_epoch  204727 non-null  float64
 2   frame.len         204727 non-null  int64  
 3   frame.cap_len     204727 non-null  int64  
 4   ip.version        204727 non-null  int64  
 5   ip.hdr_len        204727 non-null  float64
 6   ip.flags.rb       204727 non-null  float64
 7   ip.flags.df       204727 non-null  float64
 8   ip.flags.mf       204727 non-null  float64
 9   ip.frag_offset    204727 non-null  float64
 10  ip.ttl            204727 non-null  float64
 11  ip.proto          204727 non-null  float64
 12  ip.len            204727 non-null  float64
 13  srcport           204727 non-null  int64  
 14  dstport           204727 non-null  int64  
 15  length            204727 non-null  int64  
dtypes: float64(9), int64

Split train and test


In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(
    clean_dataset, targets, test_size=0.20, random_state=7, shuffle=True
)

clean_dataset.reset_index(drop=True, inplace=True)
clean_dataset.to_csv("cleaned-dataset.csv")
clean_dataset


,frame.encap_type,frame.time_epoch,frame.len,frame.cap_len,ip.version,ip.hdr_len,ip.flags.rb,ip.flags.df,ip.flags.mf,ip.frag_offset,ip.ttl,ip.proto,ip.len,srcport,dstport,length
0,1,1.645532e+09,75,75,4,20.0,0.0,1.0,0.0,0.0,128.0,17.0,61.0,55723,443,41
1,1,1.645532e+09,67,67,4,20.0,0.0,1.0,0.0,0.0,58.0,17.0,53.0,443,55723,33
2,1,1.645532e+09,75,75,4,20.0,0.0,1.0,0.0,0.0,128.0,17.0,61.0,52672,443,41
3,1,1.645532e+09,67,67,4,20.0,0.0,1.0,0.0,0.0,58.0,17.0,53.0,443,52672,33
4,1,1.645532e+09,292,292,4,20.0,0.0,1.0,0.0,0.0,255.0,17.0,278.0,5353,5353,258
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204722,1,1.645534e+09,54,54,4,20.0,0.0,1.0,0.0,0.0,128.0,6.0,40.0,49827,443,20
204723,1,1.645534e+09,66,66,4,20.0,0.0,1.0,0.0,0.0,231.0,6.0,52.0,443,49826,32
204724,1,1.645534e+09,54,54,4,20.0,0.0,1.0,0.0,0.0,128.0,6.0,40.0,49826,443,20
204725,1,1.645534e+09,263,263,4,20.0,0.0,1.0,0.0,0.0,128.0,6.0,249.0,49826,443,20


Size and Train data colunmns


In [6]:
print("Size of test: {0}, {1}".format(len(X_test), len(Y_test)))
print("Size of train: {0}, {1}".format(len(X_train), len(Y_train)))

X_train


Size of test: 40946, 40946
Size of train: 163781, 163781


,frame.encap_type,frame.time_epoch,frame.len,frame.cap_len,ip.version,ip.hdr_len,ip.flags.rb,ip.flags.df,ip.flags.mf,ip.frag_offset,ip.ttl,ip.proto,ip.len,srcport,dstport,length
117820,1,1.645532e+09,1292,1292,4,20.0,0.0,1.0,0.0,0.0,58.0,17.0,1278.0,443,56480,1258
169836,1,1.645533e+09,75,75,4,20.0,0.0,1.0,0.0,0.0,128.0,17.0,61.0,61360,443,41
110069,1,1.645532e+09,123,123,4,20.0,0.0,1.0,0.0,0.0,128.0,17.0,109.0,56480,443,89
201718,1,1.645534e+09,1292,1292,4,20.0,0.0,1.0,0.0,0.0,58.0,17.0,1278.0,443,61801,1258
186308,1,1.645534e+09,1292,1292,4,20.0,0.0,1.0,0.0,0.0,122.0,17.0,1278.0,443,61540,1258
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53459,1,1.645532e+09,1292,1292,4,20.0,0.0,1.0,0.0,0.0,58.0,17.0,1278.0,443,56480,1258
10742,1,1.645532e+09,1127,1127,4,20.0,0.0,1.0,0.0,0.0,128.0,17.0,1113.0,56453,443,1093
49689,1,1.645532e+09,1292,1292,4,20.0,0.0,1.0,0.0,0.0,58.0,17.0,1278.0,443,56480,1258
189636,1,1.645534e+09,1292,1292,4,20.0,0.0,1.0,0.0,0.0,58.0,17.0,1278.0,443,51905,1258


Logistic Regression


In [7]:
%%time

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Fitting the logistic regression algorithm to training dataset
model = LogisticRegression(random_state = 7)
clf = model.fit(X_train, Y_train)
predict = clf.predict(X_test)

print(classification_report(Y_test, predict))

print("Acuracy: {0}%".format(model.score(X_test,Y_test) * 100))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00      2687
           1       0.93      1.00      0.97     38259

    accuracy                           0.93     40946
   macro avg       0.47      0.50      0.48     40946
weighted avg       0.87      0.93      0.90     40946

Acuracy: 93.43769843208128%
CPU times: user 1.45 s, sys: 448 ms, total: 1.9 s
Wall time: 3.87 s


/Users/mutnawaz/Desktop/Muteeb/Projects/Paper/quic-classifier/quic/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/mutnawaz/Desktop/Muteeb/Projects/Paper/quic-classifier/quic/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/mutnawaz/Desktop/Muteeb/Projects/Paper/quic-classifier/quic/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_

KNN Algorithm


In [8]:
%%time

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

neigh = KNeighborsClassifier()
neigh.fit(X_train, Y_train)
predictions = neigh.predict(X_test)

print(classification_report(Y_test, predictions))

print("Acuracy: {0}%".format(neigh.score(X_test, Y_test) * 100))


Decision tree


In [ ]:
%%time

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

model = DecisionTreeClassifier(random_state=0)
model.fit(X_train, Y_train)
predictions = model.predict(X_test)

print(classification_report(Y_test, predictions))

print("Acuracy: {0}%".format(model.score(X_test, Y_test) * 100))


Random Forset


In [ ]:
%%time

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

model = RandomForestClassifier(random_state=0)
model.fit(X_train, Y_train)
predictions = model.predict(X_test)
report = classification_report(Y_test, predictions)

print(report)

print("Acuracy: {0}%".format(model.score(X_test, Y_test) * 100))


Feature importance for random forest


In [ ]:
%%time

import numpy as np
import pandas as pd

importances = model.feature_importances_
std = np.std([tree.feature_importances_ for tree in model.estimators_], axis=0)

features = list(clean_dataset.columns)
classifier_importances = pd.Series(importances, index=features)
print(classifier_importances)

fig, ax = plt.subplots()
classifier_importances.plot.bar(yerr=std, ax=ax)
ax.set_title("Feature importances")
ax.set_ylabel("Mean decrease in impurity")
fig.tight_layout(rect=(0, 0, 2, 2))
plt.show()


Plot classification.

pip3 install zip

Link: https://stackoverflow.com/questions/28200786/how-to-plot-scikit-learn-classification-report


In [ ]:
# pip3 install zip

import numpy as np
import matplotlib.pyplot as plt


def show_values(pc, fmt="%.2f", **kw):
    """
    Heatmap with text in each cell with matplotlib's pyplot
    Source: https://stackoverflow.com/a/25074150/395857
    By HYRY
    """
    pc.update_scalarmappable()
    ax = pc.axes
    # ax = pc.axes# FOR LATEST MATPLOTLIB
    # Use zip BELOW IN PYTHON 3
    for p, color, value in zip(pc.get_paths(), pc.get_facecolors(), pc.get_array()):
        x, y = p.vertices[:-2, :].mean(0)
        if np.all(color[:3] > 0.5):
            color = (0.0, 0.0, 0.0)
        else:
            color = (1.0, 1.0, 1.0)
        ax.text(x, y, fmt % value, ha="center", va="center", color=color, **kw)


def cm2inch(*tupl):
    """
    Specify figure size in centimeter in matplotlib
    Source: https://stackoverflow.com/a/22787457/395857
    By gns-ank
    """
    inch = 2.54
    if type(tupl[0]) == tuple:
        return tuple(i / inch for i in tupl[0])
    else:
        return tuple(i / inch for i in tupl)


def heatmap(
    AUC,
    title,
    xlabel,
    ylabel,
    xticklabels,
    yticklabels,
    figure_width=40,
    figure_height=20,
    correct_orientation=False,
    cmap="RdBu",
):
    """
    Inspired by:
    - https://stackoverflow.com/a/16124677/395857
    - https://stackoverflow.com/a/25074150/395857
    """

    # Plot it out
    fig, ax = plt.subplots()
    # c = ax.pcolor(AUC, edgecolors='k', linestyle= 'dashed', linewidths=0.2, cmap='RdBu', vmin=0.0, vmax=1.0)
    c = ax.pcolor(AUC, edgecolors="k", linestyle="dashed", linewidths=0.2, cmap=cmap)

    # put the major ticks at the middle of each cell
    ax.set_yticks(np.arange(AUC.shape[0]) + 0.5, minor=False)
    ax.set_xticks(np.arange(AUC.shape[1]) + 0.5, minor=False)

    # set tick labels
    # ax.set_xticklabels(np.arange(1,AUC.shape[1]+1), minor=False)
    ax.set_xticklabels(xticklabels, minor=False)
    ax.set_yticklabels(yticklabels, minor=False)

    # set title and x/y labels
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)

    # Remove last blank column
    plt.xlim((0, AUC.shape[1]))

    # Turn off all the ticks
    ax = plt.gca()
    for t in ax.xaxis.get_major_ticks():
        t.tick1On = False
        t.tick2On = False
    for t in ax.yaxis.get_major_ticks():
        t.tick1On = False
        t.tick2On = False

    # Add color bar
    plt.colorbar(c)

    # Add text in each cell
    show_values(c)

    # Proper orientation (origin at the top left instead of bottom left)
    if correct_orientation:
        ax.invert_yaxis()
        ax.xaxis.tick_top()

    # resize
    fig = plt.gcf()
    # fig.set_size_inches(cm2inch(40, 20))
    # fig.set_size_inches(cm2inch(40*4, 20*4))
    fig.set_size_inches(cm2inch(figure_width, figure_height))


def plot_classification_report(
    classification_report, title="Classification report ", cmap="RdBu"
):
    """
    Plot scikit-learn classification report.
    Extension based on https://stackoverflow.com/a/31689645/395857
    """
    lines = classification_report.split("\n")

    classes = []
    plotMat = []
    support = []
    class_names = []
    for line in lines[2 : (len(lines) - 2)]:
        t = line.strip().split()
        if len(t) < 2:
            continue
        classes.append(t[0])
        v = [float(x) for x in t[1 : len(t) - 1]]
        support.append(int(t[-1]))
        class_names.append(t[0])
        print(v)
        plotMat.append(v)

    print("plotMat: {0}".format(plotMat))
    print("support: {0}".format(support))

    xlabel = "Metrics"
    ylabel = "Classes"
    xticklabels = ["Precision", "Recall", "F1-score"]
    yticklabels = [
        "{0} ({1})".format(class_names[idx], sup) for idx, sup in enumerate(support)
    ]
    figure_width = 25
    figure_height = len(class_names) + 7
    correct_orientation = False
    heatmap(
        np.array(plotMat),
        title,
        xlabel,
        ylabel,
        xticklabels,
        yticklabels,
        figure_width,
        figure_height,
        correct_orientation,
        cmap=cmap,
    )


def main():
    sampleClassificationReport = """             precision    recall  f1-score   support

          Acacia       0.62      1.00      0.76        66
          Blossom       0.93      0.93      0.93        40
          Camellia       0.59      0.97      0.73        67
          Daisy       0.47      0.92      0.62       272
          Echium       1.00      0.16      0.28       413

        avg / total       0.77      0.57      0.49       858"""

    plot_classification_report(sampleClassificationReport)
    plt.show()
    # plt.savefig("classification_report.png", dpi=200, format="png", bbox_inches="tight")
    plt.close()


main()


Gradient boosting


In [ ]:
%%time

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report

model = GradientBoostingClassifier(random_state=0)
model.fit(X_train, Y_train)
predictions = model.predict(X_test)

print(classification_report(Y_test, predictions))

print("Acuracy: {0}%".format(model.score(X_test, Y_test) * 100))


Plot
